# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'api_keys'

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/User/Hokage/OneDrive/Desktop/Repositories/python-api/Starter_Code/starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

FileNotFoundError: [Errno 2] File C:/User/Hokage/OneDrive/Desktop/Repositories/python-api/Starter_Code/starter_Code/output_data/cities.csv does not exist: 'C:/User/Hokage/OneDrive/Desktop/Repositories/python-api/Starter_Code/starter_Code/output_data/cities.csv'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width=500,
    frame_height=400,
    geo=True,
    tiles="CartoLight",
    colorbar=True,
    clim=(0, 100),
)

# Display the map
hvplot.show(plot)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.query('`Max Temp` >= 280.00 and `Max Temp` <= 300.00 and `Wind Speed` < 1.54 and `Cloudiness` == 0')

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,santa maria,-29.6842,-53.8069,288.19,32,0,0.94,BR,1676765280
163,163,sidhi,24.4167,81.8833,289.55,48,0,0.43,IN,1676765574
181,181,hadgaon,19.5000,77.6667,293.65,27,0,1.47,IN,1676765581
196,196,tura,25.5198,90.2201,289.23,46,0,1.34,IN,1676765595
199,199,saint george,37.1041,-113.5841,287.18,27,0,1.34,US,1676765553


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
72,santa maria,BR,-29.6842,-53.8069,32,
163,sidhi,IN,24.4167,81.8833,48,
181,hadgaon,IN,19.5000,77.6667,27,
196,tura,IN,25.5198,90.2201,46,
199,saint george,US,37.1041,-113.5841,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
limit = 20
radius = 10000
params = {
    "categories" : categories,
    "limit":limit,
    "radius" :radius,
    "sort" : "distance",
    "apiKey" :geoapify_key
}
# Set the parameters for the type of search
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filters'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params).json()
    # Convert the API response to JSON format
    name_address = response
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
santa maria - nearest hotel: Hotel Morotin
sidhi - nearest hotel: No hotel found
hadgaon - nearest hotel: Hotel Krishna Palace Mahur
tura - nearest hotel: Eco Park Rest House
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
pyay - nearest hotel: Yoma Royal
dok kham tai - nearest hotel: V C Hotel
chandvad - nearest hotel: Nasik Dhuliya Road
sironj - nearest hotel: No hotel found
redlands - nearest hotel: Ayres Hotel Redlands


,City,Country,Lat,Lng,Humidity,Hotel Name
72,santa maria,BR,-29.6842,-53.8069,32,Hotel Morotin
163,sidhi,IN,24.4167,81.8833,48,No hotel found
181,hadgaon,IN,19.5000,77.6667,27,Hotel Krishna Palace Mahur
196,tura,IN,25.5198,90.2201,46,Eco Park Rest House
199,saint george,US,37.1041,-113.5841,27,"The Advenire, an Autograph Collection Hotel"
212,pyay,MM,18.8167,95.2167,65,Yoma Royal
389,dok kham tai,TH,19.1624,99.9934,68,V C Hotel
459,chandvad,IN,20.3333,74.2500,25,Nasik Dhuliya Road
462,sironj,IN,24.1000,77.7000,40,No hotel found
505,redlands,US,34.0556,-117.1825,21,Ayres Hotel Redlands


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hover_cols = ['City', 'Country', 'Hotel Name']
hover_tooltips = [(col, '@'+col) for col in hover_cols]

plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    title="City Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width=500,
    frame_height=400,
    geo=True,
    tiles="CartoLight",
    colorbar=True,
    clim=(0, 100),
    hover_cols=hover_cols,
    hover_tooltips=hover_tooltips,
)

# Display the map
hvplot.show(plot)